In [1]:
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import RMSprop
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras import layers
from keras import models
from random import randint
import math
import sys
import os

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
# path to directory
base_dir = "C:\\Users\\Cheikh\\Desktop\\projetChars74k"

fnt_base_dir = os.path.join(base_dir, "EnglishFnt")
img_base_dir = os.path.join(base_dir, "EnglishImg")
hnd_base_dir = os.path.join(base_dir, "EnglishHnd")

# utils functions used to plot random image in dataset


def showFolderImageSample(base_folder, img_number=2):
    '''
    base_folder(String) : directory in which we'll look for sample
    img_number : for each folder how many img to show
    '''
    nrows = ncols = img_number

    fig = plt.gcf()
    fig.set_size_inches(ncols * img_number, nrows * img_number)
    dir_names = os.listdir(base_folder)

    # get sample directory
    sample_dir = []
    for i in range(img_number):
        sample_dir.append(dir_names[randint(0, 61)])
    dir_names = [os.path.join(base_folder, dname) for dname in sample_dir]

    print("total sample directory : {}".format(len(dir_names)))
    # for each sample directory get img_number random image

    img = []
    for d in dir_names:
        for i in range(img_number):
            img.append(os.path.join(d, os.listdir(
                d)[randint(0, len(os.listdir(d)) - 1)]))

    for i, img_path in enumerate(img):
        # Set up subplot; subplot indices start at 1
        sp = plt.subplot(nrows, ncols, i + 1)
        sp.axis('Off')  # Don't show axes (or gridlines)
        img = mpimg.imread(img_path)
        plt.imshow(img)

    plt.show()
    print("Size of one random image : {}".format(img.shape))
    print(" Example image : \n {}".format(img[0]))

########################################################################


def dirTotalFile(base):
    fileNumber = 0
    if(os.path.isfile(base)):
        return 0
    _dir = os.listdir(base)
    for d in _dir:
        _d = os.path.join(base, d)
        if(os.path.isdir(_d)):
            fileNumber += dirTotalFile(_d)
        else:
            fileNumber += 1
    return fileNumber


########################################################################

showFolderImageSample(fnt_base_dir)


Using TensorFlow backend.


total sample directory : 2


<Figure size 400x400 with 4 Axes>

Size of one random image : (128, 128)
 Example image : 
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]


In [28]:
_list = [i for i in range(0, 10)] + [chr(i) for i in range(97, 123)] + [chr(i) for i in range(65, 91)]


class Utils:

    def __init__(self, path):
        self.path = path
        self.files = {}

    def readImage(self, start, size, _type):
        print('Reading {} dataset image....'.format(_type))
        begin = time.time()
        files = self.files
        data = []
        label = []
        j = 0
        #
        for key in files.keys():
            for i in range(start, start+size):
                label.append(key)
                data.append(plt.imread(files[key][i]))
            j = j + 1.613
            if(j > 100):
                j = 100
            sys.stdout.write("\r{0:.2f}%".format(j))
            sys.stdout.flush()
        end = time.time()
        le = LabelEncoder()
        label = le.fit_transform(label)
        label = to_categorical(label)

        print(" \n Time for reading {} images : {}".format(_type, end-begin))
        return np.array(data), label

    def loadData(self):

        train_images_size = 10 #775
        test_size =  3 #241

        path = self.path
        files = self.files

        if(not(os.path.exists(path))):
            raise("Directory doesn't exist")

        dnames = [os.path.join(path, dname) for dname in os.listdir(path)]

        # read all the image
        for i, label in enumerate(_list):
            if(not(os.path.isdir(dnames[i]))):
                continue
            files[label] = [os.path.join(dnames[i], f) for f in os.listdir(dnames[i])]
        
        train, train_label = self.readImage(0, train_images_size, "training")
        
        test, test_label = self.readImage(train_images_size, test_size, "test")
        
        return (train, train_label),(test, test_label)


utils = Utils(fnt_base_dir)
(train, train_label),(test, test_label) = utils.loadData()


Reading training dataset image....
100.00% 
 Time for reading training images : 1.3115715980529785
Reading test dataset image....
100.00% 
 Time for reading test images : 0.4641077518463135


In [29]:
# keras font model

print(train[0].shape)



epochs = 10
nodes = 32
optimizer = 'rmsprop'



network = models.Sequential()
network.add(layers.Dense(32,activation = 'relu', input_shape = (128,128,1)))
network.add(layers.Flatten())
network.add(layers.Dense(62,activation='softmax'))
network.summary()


(128, 128)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 128, 128, 32)      64        
_________________________________________________________________
flatten_6 (Flatten)          (None, 524288)            0         
_________________________________________________________________
dense_22 (Dense)             (None, 62)                32505918  
Total params: 32,505,982
Trainable params: 32,505,982
Non-trainable params: 0
_________________________________________________________________


In [25]:

network.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['acc'])


history = network.fit(train, train_label, batch_size=32,
                      epochs=epochs)

history_dict = history.history
loss_values = history_dict['loss']
acc_values = history_dict['acc']
plt.rcParams['figure.figsize'] = (16, 8)  # Make the figures a bit bigger
fig, (ax1, ax2,) = plt.subplots(1, 2)

x = range(1, epochs+1)
ax1.plot(x, loss_values, 'bo', label='Training Loss')
ax1.set_title('Training loss ')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.legend()


ax2.plot(x, acc_values, 'bo', label='Training Accuracy')
ax2.set_title('Training accuracy ')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax2.legend()

plt.show()

ValueError: Error when checking input: expected dense_15_input to have 4 dimensions, but got array with shape (48050, 16384, 1)

In [ ]:
test_loss, test_acc = network.evaluate(test,test_label)
print('accuracy on test set : {}'.format(test_acc))

In [ ]:
prediction = network.predict_classes(test,verbose=1)
